In [ ]:
import shutil
from pathlib import Path
from typing import cast

import lief
import pefile
import polars as pl
from tqdm import tqdm

# Paths

Everything here is done on a simple dllhost exe `fake_neomon_host/host-bind.exe`

Dumps are made once the _DllEntryPoint is reached (0x13903)

In [ ]:
base = Path("../neomon-dump/dumps")
base_patch = Path("../neomon-dump/patches")

In [ ]:
# imports csv

dump_imports_p = base / "dump-imports.csv"
old_iat_p = base / "old-iat.csv"
byte_calls_p = base / "broken-byte-calls.csv"
module_imports_p = base / "../NeoMon.dll.export.full.csv"

In [ ]:
# exports csv
patch_thunks_p = base_patch / "thunks_patch.csv"
patch_calls_p = base_patch / "calls_patch.csv"
patch_iat_p = base_patch / "iat_patch.csv"

In [ ]:
base_to_exe = Path('../fake_neomon_host')
original_dump_path = base_to_exe / "NeoMon_dump.dll"
patched_path = base_to_exe / "NeoMon_patched.dll"

In [ ]:
fake_sections = range(213, 217)

In [ ]:
from addr_helpers import hex_to_LE, rel_call, to_bin

patch_schema = {
    "patch_addr": pl.String,
    "mem_old": pl.String,
    "patch": pl.String,
}

# Parsing import table

In [ ]:
dump_imports = pl.read_csv(dump_imports_p)
dump_imports.columns = [
    "Address",
    "Type",
    "Ordinal",
    "Symbol",
    "undecorated",
]
dump_imports = dump_imports.with_columns(pl.when(pl.col("Type") == "Экспорт").then(pl.lit("Export").alias("Type")).otherwise(pl.lit("Import").alias("Type")))
dump_imports = dump_imports.filter(pl.col("Type") == "Export").drop("Type", "undecorated").rename({"Symbol": "Function"})

module_imports = pl.read_csv(module_imports_p).drop("Function")
dump_imports = dump_imports.join(module_imports, on='Address')
dump_imports = dump_imports.filter(~pl.col("Module").is_in(["game.exe", "host-bind.exe"]))

assert dump_imports.filter(pl.col("Module").is_null()).shape[0] == 0

print(dump_imports.shape)

In [ ]:
dump_imports = dump_imports.with_columns(("0x" + pl.col("Address").str.to_lowercase()).alias("Address"))
dump_imports = dump_imports.unique("Address", keep='first').sort("Address")

In [ ]:
dump_imports.head()

# Gathering imports from old IAT

In [ ]:
iat = pl.read_csv(old_iat_p)

iat = iat.rename({"Address": "Calladdr", "Destination": "Address"})
iat = iat.with_columns(("0x" + pl.col("Address").str.to_lowercase()).alias("Address"))

iat = iat.join(dump_imports, on='Address', how='left')

In [ ]:
for i in fake_sections:
    modname = f"section_{i}"
    iat = iat.with_columns(pl.when(pl.col("Address").str.starts_with(f'0x0{i}')).then(pl.lit(modname)).otherwise("Module").alias("Module"))

Make sure iat2 is empty, i.e. no unknown calls present

In [ ]:
iat2 = iat.filter(pl.col("Module").is_null())
iat2 = iat2.filter(pl.col("Address").str.slice(2).str.to_integer(base=16) != 0)
iat2

Cancel forwarding imports (e.g. kernel32.dll,AllocateHeap -> ntdll.dll,RtlReAllocateHeap)

In [ ]:
systemroot = "C:/Windows/System32/"
forwarding_modules = ["kernel32.dll", "user32.dll"]

unforward_map: dict[str, tuple[str, str]] = dict()

forwarded = iat.filter(pl.col("Module") == "ntdll.dll")

for modname in forwarding_modules:
    modpath = systemroot + modname
    number = 0

    dll = pefile.PE(modpath)
    dll.full_load()
    for exp in dll.DIRECTORY_ENTRY_EXPORT.symbols:
        name = exp.name.decode() if exp.name else f"Ordinal#{exp.ordinal}"
        forward_to = ''
        if exp.forwarder:
            forward_to = exp.forwarder.decode().removeprefix("NTDLL.")
        if forwarded.filter(pl.col("Function") == forward_to).shape[0] > 0:
            number += 1
            unforward_map[forward_to] = (modname, name)
    
    print(f'For {modname} there are {number} forwards')

In [ ]:
for func in iat.filter(pl.col("Module") == "ntdll.dll")["Function"]:
    if func not in unforward_map:
        print(f"Func {func} from ntdll.dll is not found in forward map")
        continue

    origmod, origfunc = unforward_map[func]

    if "InitializeCrit" in func:
        print(func, origmod, origfunc)

    condition = (pl.col("Module") == "ntdll.dll") & (pl.col("Function") == func)
    iat = iat.with_columns(
        [
            pl.when(condition)
            .then(pl.lit(origmod))
            .otherwise("Module")
            .alias("Module"),
            pl.when(condition)
            .then(pl.lit(origfunc))
            .otherwise("Function")
            .alias("Function"),
        ]
    )

In [ ]:
iat.write_csv(str(old_iat_p) + '2.csv')

In [ ]:
iat_seg = (
    iat.sort("Calladdr").fill_null("")
    .with_columns(
        (pl.col("Module") != pl.col("Module").shift(1)).cum_sum().alias("segment_id")
    )
    .fill_null(0)
    .filter(pl.col("Address").str.slice(2).str.to_integer(base=16) != 0)
    .filter(pl.col("Module") != '')
    .filter(~pl.col("Module").str.starts_with("section_"))
)
segments = [group.drop("segment_id") for _, group in iat_seg.group_by("segment_id", maintain_order=True)]

In [ ]:
# extract obfuscated imports for later
obfuscated = iat.filter(pl.col("Module").str.starts_with('section_')).filter(pl.col("Address").str.slice(2).str.to_integer(base=16) != 0)
obfuscated.head(3)

In [ ]:
# remove gaps and one obfuscated import
w = iat.shape[0]
iat = iat.filter(pl.col("Module").is_not_null())
print(f"Filtered out {iat.shape[0]}/{w} iat entries")

In [ ]:
# confirm all names are decorated
assert iat.filter(pl.col("Function").str.contains("public")).shape[0] == 0

# Move fake sections to the end

In [ ]:
shutil.copy(original_dump_path, patched_path)

pe_lief = cast(lief.PE.Binary, lief.PE.parse(patched_path))
pe_lief.remove_all_imports()

In [ ]:
def get_data(sec_i: int) -> bytes:
    return open(base_to_exe / f'neomon{sec_i}.bin', 'rb').read()

In [ ]:
for sec_i in fake_sections:
    sec = lief.PE.Section(f'.fake{sec_i}')
    data = get_data(sec_i)
    sec.content = memoryview(data)

    sec.virtual_size = len(data)

    CH = lief.PE.Section.CHARACTERISTICS
    # sec.characteristics = int(CH.MEM_READ | CH.MEM_WRITE | CH.MEM_EXECUTE | CH.CNT_INITIALIZED_DATA)
    sec.characteristics = int(CH.MEM_EXECUTE | CH.CNT_INITIALIZED_DATA)

    pe_lief.add_section(sec)
    
    sec = pe_lief.get_section(f'.fake{sec_i}')
    if sec is None:
        print("Error: failed to add section")

In [ ]:
config = lief.PE.Builder.config_t()
config.imports = True

bb = lief.PE.Builder(pe_lief, config)
bb.build()
bb.write(str(patched_path))

# Constructing new IDT

In [ ]:
def create_32bit_ordinal_import(ordinal_number: int) -> lief.PE.ImportEntry:
    """
    Create a 32-bit import by ordinal

    Args:
        ordinal_number: The ordinal number (0-65535)
    """
    # Validate ordinal range
    if ordinal_number < 0 or ordinal_number > 0xFFFF:
        raise ValueError("Ordinal number must be between 0 and 65535")

    # For 32-bit PE:
    # - Set bit 31 to 1 (0x80000000)
    # - Bits 30-16 must be 0
    # - Bits 15-0 contain the ordinal
    ORDINAL_MASK_32 = 0x80000000
    data_value = ORDINAL_MASK_32 | ordinal_number

    # Create the import entry
    entry = lief.PE.ImportEntry(data_value, lief.PE.PE_TYPE.PE32)

    return entry

In [ ]:
from pydantic import BaseModel


class Sect(BaseModel):
    name: str
    raw_addr: int
    raw_size: int
    virt_addr: int
    virt_size: int
    chars: int

    @staticmethod
    def from_section(section: lief.PE.Section) -> "Sect":
        return Sect(
            name=section.name,
            raw_addr=section.offset,
            raw_size=section.size,
            virt_addr=section.virtual_address,
            virt_size=section.virtual_size,
            chars=section.characteristics,
        )

    def to_section(self) -> lief.PE.Section:
        sect = lief.PE.Section(self.name)
        sect.name = self.name
        sect.offset = self.raw_addr
        sect.size = self.raw_size
        sect.virtual_address = self.virt_addr
        sect.virtual_size = self.virt_size
        sect.characteristics = self.chars
        return sect

In [ ]:
# adding proper .idata
# new_idata = lief.PE.Section(".idata")
# new_idata.offset = idata_offset
# new_idata.size = idata_size
# new_idata.virtual_address = idata_virtual_address - pe_lief.imagebase
# new_idata.virtual_size = idata_virtual_size
# new_idata.characteristics = pe_lief.sections[0].characteristics
# pe_lief.add_section(new_idata)

# sections = [Sect.from_section(sec) for sec in pe_lief.sections]
# sections = [sections[0], Sect.from_section(new_idata)] + sections[1:]

# N = len(pe_lief.sections)
# for i in range(N):
#     pe_lief.sections[i].name = f'{i}'

# for i in range(N):
#     pe_lief.remove_section(f'{i}')

# for sec in sections:
#     pe_lief.add_section(sec.to_section())

In [ ]:
pe_lief = cast(lief.PE.Binary, lief.PE.parse(patched_path))

In [ ]:
for s in pe_lief.sections:
    print(s.name, hex(s.virtual_address), hex(s.virtual_address + s.virtual_size))

### Adding imports

Creates brand new IDT with new IAT and ILT

In [ ]:
for seg in segments:
    dll = seg['Module'][0]
    if dll is None or dll == '':
        continue

    mod = pe_lief.add_import(dll)
    for calladdr, addr, ordinal, func, mname in seg.rows():
        if func.startswith('Ordinal#'):
            # ordinal = int(func.removeprefix("Ordinal#"))
            entry = create_32bit_ordinal_import(ordinal)
        else:
            entry = lief.PE.ImportEntry(func)
        mod.add_entry(entry)

In [ ]:
config = lief.PE.Builder.config_t()
config.imports = True

bb = lief.PE.Builder(pe_lief, config)
bb.build()
bb.write(str(patched_path))

Reset IAT to the old IAT address

In [ ]:
pe = pefile.PE(patched_path)
pe.full_load()

In [ ]:
assert len(pe.DIRECTORY_ENTRY_IMPORT) == len(segments), "Change the MAX_REPEATED_ADDRESSES to >20" # type: ignore

In [ ]:
for i, seg in enumerate(segments):
    first_thunk = int(seg["Calladdr"][0], 16)

    pe.DIRECTORY_ENTRY_IMPORT[i].struct.FirstThunk = ( # type: ignore
        first_thunk - pe.OPTIONAL_HEADER.ImageBase # type: ignore
    )

### Fixing IAT to the fake sections

In [ ]:
new_fake_sections = pl.DataFrame(
    [
        {
            "Module": s.Name.decode().replace(".fake", "section_"),
            "NAddress": hex(s.VirtualAddress + pe.OPTIONAL_HEADER.ImageBase),  # type: ignore
            "OAddress": f"0x0{s.Name.decode().replace('.fake', '')}0000",
        }
        for s in pe.sections
        if s.Name.decode().startswith(".fake")
    ]
)

In [ ]:
obfuscated_iat = obfuscated.join(new_fake_sections, on='Module').drop("Ordinal", "Function")
obfuscated_iat.head(3)

In [ ]:
iat_patch = pl.DataFrame(schema=patch_schema)
for calladdr, addr, module, new_addr, old_addr in obfuscated_iat.rows():
    offset = int(new_addr, 16) - int(old_addr, 16)
    naddr = int(addr, 16) + offset
    iat_patch = iat_patch.vstack(pl.DataFrame({
        'patch_addr': calladdr,
        'mem_old': to_bin(hex_to_LE(int(addr, 16))),
        'patch': to_bin(hex_to_LE(naddr))
    }))

print(iat_patch.shape)
iat_patch.head(3)

In [ ]:
iat_patch.write_csv(patch_iat_p)

### Saving stuff

In [ ]:
temp = "tmp"
pe.write(filename=temp)
pe.close()
shutil.move(temp, patched_path)

# Fix thunks

Suspect thunks are searched via <90 e9 ? ? ? ?> wildcard

In [ ]:
thunks = pl.read_csv(byte_calls_p)
thunks = thunks.filter(pl.col("Instruction").is_in(["jmp"]))
thunks = thunks.with_columns(pl.col("Call address").str.slice(2).str.to_integer(base=16).alias("Int_addr"))
thunks = thunks.filter(pl.col("Int_addr") < 0x01588000)  # this is where the old IAT begins
thunks = thunks.drop('subroutine', 'Instruction', 'Resolved name')
thunks.shape

In [ ]:
valid_addresses = set(dump_imports["Address"].to_list())
thunks = thunks.filter(pl.col("Destination").is_in(valid_addresses))
print(thunks.shape)

jmp is considered a thunk as long as it has at least two neighbouring jmps. Both neighbours would be counted as thunks as well.

In [ ]:
get_addr = pl.col("Call address").str.slice(2).str.to_integer(base=16)
is_prev = pl.col("Int_addr").shift(1) + 6 == pl.col("Int_addr")
is_next = pl.col("Int_addr").shift(-1) - 6 == pl.col("Int_addr")

# thunks = thunks.with_columns([is_prev.alias('is_prev'), is_next.alias('is_next')])

thunks = thunks.with_columns(get_addr.alias("Int_addr"))
thunks = thunks.with_columns((is_next | is_prev).alias("is_thunk"))
thunks = thunks.with_columns(pl.col("is_thunk") | pl.col("is_thunk").shift(1) | pl.col("is_thunk").shift(-1))
thunks.filter("is_thunk").shape[0]

In [ ]:
thunks.filter(abs(pl.col("Int_addr") - 0x14ab766) < 48)

In [ ]:
# additional thunks
thunks = thunks.with_columns(pl.col("is_thunk") | (pl.col("Call address") == "0x14aaadb"))
thunks = thunks.with_columns(pl.col("is_thunk") | (pl.col("Call address") == "0x14aaad5"))

thunks = thunks.with_columns(pl.col("is_thunk") | (pl.col("Call address") == "0x14aa9f5"))
thunks = thunks.with_columns(pl.col("is_thunk") | (pl.col("Call address") == "0x14aa9fb"))

thunks = thunks.with_columns(pl.col("is_thunk") | (pl.col("Call address") == "0x14aab3d"))

thunks = thunks.with_columns(pl.col("is_thunk") | (pl.col("Call address") == "0x14ab35d"))
thunks = thunks.with_columns(pl.col("is_thunk") | (pl.col("Call address") == "0x14ab363"))

thunks = thunks.with_columns(pl.col("is_thunk") | (pl.col("Call address") == "0x14ab405"))
thunks = thunks.with_columns(pl.col("is_thunk") | (pl.col("Call address") == "0x14ab40b"))

thunks = thunks.with_columns(pl.col("is_thunk") | (pl.col("Call address") == "0x14ab767"))
thunks = thunks.with_columns(pl.col("is_thunk") | (pl.col("Call address") == "0x14ab76d"))

thunks = thunks.with_columns(pl.col("is_thunk") | (pl.col("Call address") == "0x14abc33"))
thunks = thunks.with_columns(pl.col("is_thunk") | (pl.col("Call address") == "0x14abc39"))

thunks = thunks.with_columns(pl.col("is_thunk") | (pl.col("Call address") == "0x14abe61"))

thunks = thunks.with_columns(pl.col("is_thunk") | (pl.col("Call address") == "0xcedcb1"))

thunks = thunks.with_columns(pl.col("is_thunk") | (pl.col("Call address") == "0xcedcf1"))

In [ ]:
thunks.filter(pl.col("Call address").str.slice(2).str.to_integer(base=16) == 0x14ab766)

In [ ]:
thunks = thunks.filter("is_thunk").drop("Int_addr")
print(f"Found {thunks.filter("is_thunk").unique("Destination").shape[0]} thunks and {iat.unique("Address").shape[0]} imports")

In [ ]:
thunks = thunks.join(iat.select("Module", "Function", "Address"), left_on="Destination", right_on="Address", how='left')
print(thunks.shape)
thunks.filter(pl.col("Module").is_null())

In [ ]:
# For the rest, we'll create new thunks
available_thunk_places = [
    ["0x014AB2A4", 12],
    ["0x014AB368", 8],
    ["0x014AB3C2", 14],
    ["0x014AB5B6", 10],
    ["0x014AB602", 14],
]

def find_next_addr(size: int = 6) -> str | None:
    global available_thunk_places
    for i in range(len(available_thunk_places)):
        e = available_thunk_places[i]
        if e[1] > size:
            e[1] -= size
            retval = int(e[0], 16)
            e[0] = hex(retval + size)
            return hex(retval)
    return None

In [ ]:
# Trying least intervention: no thunks = they aren't needed
skip = True

to_thunk = iat.vstack(obfuscated)
unfound = to_thunk.filter(
    ~pl.col("Address").is_in(thunks.filter("is_thunk")["Destination"].to_list())
).unique("Address")
print(unfound.shape)

if skip:
    unfound = unfound.clear()

In [ ]:
thunks = thunks.with_columns(pl.lit(False).alias("new"))
for dest in unfound["Address"]:
    addr = find_next_addr()
    print(f"Using {addr} to place a new thunk to {dest}")
    if addr is None:
        raise RuntimeError("Can't place new thunk: no available space")

    func = iat.filter(pl.col("Address") == dest)
    if func.shape[0] > 0:
        modname = func["Module"][0]
        funname = func["Function"][0]
    else:
        modname, funname = None, None

    thunks = thunks.vstack(
        pl.DataFrame(
            {
                "Call address": hex(int(addr, 16)),
                "Destination": hex(int(dest, 16)),
                "is_thunk": True,
                "Module": modname,
                "Function": funname,
                "new": True,
            }
        )
    )

unfound = unfound.clear()
print(thunks.shape)

In [ ]:
# setting thunks to renewed iat
thunks = thunks.join(to_thunk.select("Calladdr", "Address").rename({"Calladdr": "iat_addr"}).unique("Address"), left_on="Destination", right_on="Address", how='left')
thunks.shape

In [ ]:
thunks.filter(pl.col("Call address").str.slice(2).str.to_integer(base=16) == 0x14ab766)

In [ ]:
thunks.tail(6)

# Fix calls

Suspect calls are <90 e8 ? ? ? ?> and <e8 ? ? ? ? 90>. The only concerning calls are "optimized". The rest point to thunks (restored) or iat (restored).

Broken jmp on 0xbd0394 (LeaveCriticalSection) made me to include nop-jumps to the list: <90 e9 ? ? ? ?>

In [ ]:
calls = pl.read_csv(byte_calls_p).filter(pl.col("Instruction").is_in(["call", "call2", "jmp"]))

# remove nop-jmps that are thunks:
thunk_addrs = set(thunks["Call address"].to_list())
calls = calls.filter(~pl.col("Call address").is_in(thunk_addrs))

# calls_inst = pl.read_csv(inst_calls_p).clear() # deprecated
# calls = calls.vstack(calls_inst).unique("Call address")
calls = calls.drop("Resolved name")
calls = calls.with_columns(pl.col("Call address").str.slice(2).str.to_integer(base=16).alias("Int_addr"))
calls = calls.filter(pl.col("Int_addr") < 0x01588000)  # this is where the old IAT begins

print(calls.shape)
calls.head(3)

In [ ]:
# remove dupes (when both patterns hit)
addrs = set(calls['Int_addr'].to_list())
calls = calls.filter(~(pl.col("Int_addr") - 1).is_in(addrs)).drop("Int_addr")

print(calls.shape)

In [ ]:
# filter only calls that point to api calls directly (still rel32 though)
valid_addresses = set(dump_imports["Address"].to_list())
calls = calls.filter(pl.col("Destination").is_in(valid_addresses))

calls.shape

In [ ]:
iat.head(1)

In [ ]:
# map with thunk addresses
thunks_to_join = thunks.select(pl.col("Destination"), pl.col("Call address").alias("thunk address")).unique("Destination")
calls = calls.join(thunks_to_join, on="Destination", how='left')

iats_to_join = iat.select(pl.col("Address").alias("Destination"), pl.col("Calladdr").alias("iat address")).unique("Destination")
calls = calls.join(iats_to_join, on="Destination", how='left')

calls.shape

In [ ]:
# all calls have their thunk and iat
unthunked = calls.filter(pl.col("thunk address").is_null()).shape[0]
print('Unthunked:', unthunked)
# disabled, since we're binding to IAT now
# assert unthunked == 0, unthunked

uniated = calls.filter(pl.col("iat address").is_null()).shape[0]
print('Uniated:', uniated)
assert uniated == 0, uniated

# Patch PE

In [ ]:
def patch_thunk(addr: str, dest: str, iat: str, new: bool, real: bool = True) -> pl.DataFrame:
    patch_addr = hex(int(addr, 16) + int(new))
    naddr = hex(int(addr, 16) + 6) # nop, jmp is 6 bytes
    mem_old = 'CC' * 6
    if not new:
        mem_old = '90E9' + to_bin(rel_call(naddr, dest))
    
    if real:
        patch = 'FF25' + to_bin(hex_to_LE(int(iat, 16)))
    else:
        patch = '90E9' + to_bin(rel_call(naddr, dest))
    
    return pl.DataFrame({
        "patch_addr": patch_addr,
        "mem_old": mem_old,
        "patch": patch,
    })

In [ ]:
thunks_patch = pl.DataFrame(schema=patch_schema)

for addr, dest, mod, new, iat_addr in thunks.drop("is_thunk", "Function").rows():
    real = mod is not None
    thunks_patch = thunks_patch.vstack(patch_thunk(addr, dest, iat_addr, new, real))
print(thunks_patch.shape)

In [ ]:
calls.head(5)

In [ ]:
calls.filter(pl.col("Call address").is_in(["0x62001b"]))

In [ ]:
# These are used in integrity checks, and need to be patched carefully
call_exceptions = [
    "0x61fffa", # Aphrodita integrity check, checks for 0x90 as first byte
]

In [ ]:
def patch_call(addr: str, inst: str, thunk_addr: str, iat_addr: str) -> dict[str, str]:
    naddr = hex(int(addr, 16) + 6)  # both nop, call and call, nop are 6-bytes
    if inst[-1] == "2":
        oaddr = hex(int(addr, 16) + 6)  # call, nop, in this case, jump starts from nop
    else:
        oaddr = naddr  # nop, call, in this case, jump starts from next instruction

    dest_rbin = to_bin(rel_call(oaddr, dest))  # rel32
    # thunk_bin = to_bin(rel_call(naddr, hex(int(thunk_addr, 16) - 5))) # TODO: wtf - 5?
    iat_rbin = to_bin(rel_call(naddr, iat_addr))  # rel32
    iat_bin = to_bin(hex_to_LE(int(iat_addr, 16)))  # imm32

    # Determine opcode based on instruction
    match inst:
        case "call":
            mem_old = "90E8" + dest_rbin
            patch = "FF15" + iat_bin
        case "jmp":
            mem_old = "90E9" + dest_rbin
            patch = "FF25" + iat_bin
            # raise RuntimeError("Shouldn't patch jumps for now")
        case _:
            raise RuntimeError(f"Unsupported instruction {inst}")
    
    if addr in call_exceptions:
        print(f"Exception call at {addr}")
        thunk_rbin = to_bin(rel_call(naddr, thunk_addr))
        patch = "90E8" + thunk_rbin
    
    return {
        "patch_addr": addr,
        "mem_old": mem_old,
        "patch": patch,
    }

In [ ]:
patch_data = []

for call in tqdm(calls.rows()):
    sub, inst, addr, dest, thunk_addr, iat_addr = call

    patch_data.append(patch_call(addr, inst, thunk_addr, iat_addr))

# Create DataFrame in one operation
calls_patch = pl.DataFrame(patch_data, schema=patch_schema).sort("patch_addr")
calls_patch.shape

In [ ]:
thunks_patch.write_csv(patch_thunks_p)
calls_patch.write_csv(patch_calls_p)

Now run ida_patch.py script in IDA Pro and apply changes

# Troubleshooting

In [ ]:
print("OK!")